In [93]:
import threading
import pyttsx3
engine = pyttsx3.init()
engine.say('测试')
engine.say('测试')
engine.startLoop(False)
Server_up = threading.Thread(target = Comm_Connection)
Server_up.start()
engine.runAndWait() 
engine.endLoop()


NameError: name 'Comm_Connection' is not defined

In [100]:
import threading
import pyttsx3
if engine._inLoop:
    engine.endLoop()
engine = pyttsx3.init()
engine.say('测试')
engine.say('测试')
engine.runAndWait() 


In [95]:
engine.endLoop()

In [175]:
import tushare as ts
import time
import pyttsx3
hl_dic = {'601595':[17,12.88]}
pos_dic = {'601595':0}

alarm_dic = hl_dic.copy()
price_dict = hl_dic.copy()
for key in alarm_dic:
    alarm_dic[key] = 0
for key in price_dict:
    price_dict[key] = {'init':0,'win':[0,0,0,0,0],'loss':[0,0,0,0,0],'to_loss':[0,0,0,0,0],"min_los":0,'max_win':0}
while True:
    
    for key,value in hl_dic.items():
        print('------------------------------------------------------------------')
        # 获取价格
        try:
            df = ts.get_realtime_quotes(key) #Single stock symbol
        except:
            print('实时数据获取失败')
            break
        dt = df[['code','name','price','bid','ask','volume','amount','time']]
        current_price = dt.price[0]
        name = dt.name[0]
        time1 = dt.time[0]
        try:
            de = ts.get_today_ticks(key)[-20:]
        except:
            print('分笔价格数据获取失败')
            break
        pre_price = de.price.values[-5]
        print(name + '  ' +key+' '+ '价格：' + str(current_price) + '  之前价格: ' + str(pre_price)+'  时间: ' + str(time1))
        
        # 首次买入提醒
        if pos_dic[key]==0:
            if float(current_price) >= value[1] and float(current_price) < value[1]*1.03 and pre_price < value[1]:
                if engine._inLoop:
                    engine.endLoop()
                engine = pyttsx3.init()
                engine.say(name+'已越过最低价,买入   '+'价格：' + str(current_price))
                engine.say(name+'已越过最低价,买入   '+'价格：' + str(current_price))
                engine.runAndWait()  
                print('首次买入声音信号已发送.................')
        else:
            # 构建所有止盈止损字典
            price_dict[key]['init'] = value[1]
            price_dict[key]['win'] = [round(value[1]*1.03,4),round(value[1]*1.05,4),round(value[1]*1.07,4),round(value[1]*1.09,4),round(value[1]*1.11,4)]
            price_dict[key]['loss'] = [round(value[1]*0.99,4),round(value[1]*1.02,4),round(value[1]*1.05,4),round(value[1]*1.08,4),round(value[1]*1.10,4)]
            price_dict[key]['min_los'] = value[1]*0.85
            price_dict[key]['max_win'] = (value[0]+value[1])/2*0.98
            
            # 录入是否到达阶段最大值
            if len(de[de.price >= price_dict[key]['win'][0]]) > 0:
                price_dict[key]['to_loss'][0] = 1
                if len(de[de.price >= price_dict[key]['win'][1]]) > 0:
                    price_dict[key]['to_loss'][1] = 1
                    if len(de[de.price >= price_dict[key]['win'][2]]) > 0:
                        price_dict[key]['to_loss'][2] = 1
                        if len(de[de.price >= price_dict[key]['win'][3]]) > 0:
                            price_dict[key]['to_loss'][3] = 1
                            if len(de[de.price >= price_dict[key]['win'][4]]) > 0:
                                price_dict[key]['to_loss'][4] = 1
            print(key,price_dict[key])
            # 判断提醒
            if float(current_price) > price_dict[key]['max_win']:
                if pre_price < price_dict[key]['max_win']:
                    if engine._inLoop:
                            engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('首次买入声音信号已发送.................')
                print('已到达终极止盈')
            else:
                if float(current_price) < price_dict[key]['min_los'] and price_dict[key]['to_loss'][0]==0:
                    if pre_price > price_dict[key]['min_los']:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                        engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('终极止声音信号已发送.................')
                    print('已到达终极止损')
                elif float(current_price) < price_dict[key]['loss'][0] and price_dict[key]['to_loss'][0]==1:
                    if pre_price > price_dict[key]['loss'][0]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之3一级止盈声音信号已发送.................')
                    print('已到达百分之3一级止盈')
                elif float(current_price) < price_dict[key]['loss'][1] and price_dict[key]['to_loss'][1]==1:
                    if pre_price > price_dict[key]['loss'][1]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之5二级止盈声音信号已发送.................')
                    print('已到达百分之5二级止盈')
                elif float(current_price) < price_dict[key]['loss'][2] and price_dict[key]['to_loss'][2]==1:
                    if pre_price > price_dict[key]['loss'][2]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之7三级止盈声音信号已发送.................')
                    print('已到达百分之7三级止盈')
                elif float(current_price) < price_dict[key]['loss'][3] and price_dict[key]['to_loss'][3]==1:
                    if pre_price > price_dict[key]['loss'][3]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之9四级止盈声音信号已发送.................')
                    print('已到达百分之9四级止盈')
                elif float(current_price) < price_dict[key]['loss'][4] and price_dict[key]['to_loss'][4]==1:
                    if pre_price > price_dict[key]['loss'][4]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之11五级止盈声音信号已发送.................')
                    print('已到达百分之11五级止盈')
    time.sleep(10)

------------------------------------------------------------------
上海电影  601595 价格：12.880  之前价格: 12.86  时间: 15:00:02
首次买入声音信号已发送.................
------------------------------------------------------------------
上海电影  601595 价格：12.880  之前价格: 12.86  时间: 15:00:02
首次买入声音信号已发送.................
------------------------------------------------------------------
上海电影  601595 价格：12.880  之前价格: 12.86  时间: 15:00:02
首次买入声音信号已发送.................


KeyboardInterrupt: 

In [41]:
import tushare as ts
import time
import pyttsx3
hl_dic = {'601595':[13.90,12.52]}
pos_dic = {'601595':1}

alarm_dic = hl_dic.copy()
for key in alarm_dic:
    alarm_dic[key] = 0

def alarm(alarm,name,str1,current_price):
    engine = pyttsx3.init()
    engine.say(name+str1+'价格：' + str(current_price))
    engine.say(name+str1+'价格：' + str(current_price))
    engine.runAndWait()  
    print(str(alarm)+'声音信号已发送.................')
while True:
    
    for key,value in hl_dic.items():
        try:
            df = ts.get_realtime_quotes(key) #Single stock symbol
        except:
            print('实时数据获取失败')
            break
        dt = df[['code','name','price','bid','ask','volume','amount','time']]
        current_price = dt.price[0]
        name = dt.name[0]
        time1 = dt.time[0]
        try:
            de = ts.get_today_ticks(key)[-20:]
        except:
            print('分比价格数据获取失败')
            break
        pre_price = de.price.values[-5]
        print(name + '  ' + '价格：' + str(current_price) + '  之前价格: ' + str(pre_price)+'  时间: ' + str(time1))
        if float(current_price) > value[1] and float(current_price) < value[1]*1.03 and pre_price < value[1]:
            alarm_dic[key] = 1
            if pos_dic[key]==0:
                alarm(alarm_dic[key],name,'已越过最低价,买入',current_price)
        elif float(current_price) > value[1]*1.03 and float(current_price) < value[1]*1.05 and pre_price < value[1]*1.03:
            alarm_dic[key] = 2
            if pos_dic[key]==1:
                alarm(alarm_dic[key],name,'已越过百分之3',current_price)
        elif float(current_price) > value[1]*1.05 and float(current_price) < value[1]*1.07 and pre_price < value[1]*1.05:
            alarm_dic[key] = 3
            if pos_dic[key]==1:
                alarm(alarm_dic[key],name,'已越过百分之5',current_price)
        elif float(current_price) > value[1]*1.07 and float(current_price) < value[1]*1.09 and pre_price < value[1]*1.07:
            alarm_dic[key] = 4
            if pos_dic[key]==1:
                alarm(alarm_dic[key],name,'已越过百分之7',current_price)
        elif float(current_price) > value[1]*1.09 and float(current_price) < value[1]*1.11 and pre_price < value[1]*1.09:
            alarm_dic[key] = 5
            if pos_dic[key]==1:
                alarm(alarm_dic[key],name,'已越过百分之9',current_price)
        elif float(current_price) > value[1]*1.11 and pre_price < value[1]*1.11:
            alarm_dic[key] = 6
            if pos_dic[key]==1:
                alarm(alarm_dic[key],name,'已越过百分之11',current_price)
        if float(current_price) > (value[0]+value[1])/2*0.98 and (value[0]+value[1])/2*0.98:
            alarm(11111,name,'到达终极止损价，止损',current_price)
        
        if alarm_dic[key] == 1 and pos_dic[key]==1:
            if float(current_price) < value[1]*0.85 and pre_price > value[1]*0.85:
                alarm(alarm_dic[key],name,'到达终极止损价，止损',current_price)
        elif alarm_dic[key]==2 and pos_dic[key]==1:
            if float(current_price) < value[1]*0.99 and pre_price > value[1]*0.99:
                alarm(alarm_dic[key],name,'到达一级止赢价，止赢',current_price)
        elif alarm_dic[key]==3 and pos_dic[key]==1:
            if float(current_price) < value[1]*1.02 and pre_price > value[1]*1.02:
                alarm(alarm_dic[key],name,'到达二级止赢价，止赢',current_price)
        elif alarm_dic[key] == 4 and pos_dic[key]==1:
            if float(current_price) < value[1]*1.05 and pre_price > value[1]*1.05:
                alarm(alarm_dic[key],name,'到达三级止赢价，止赢',current_price)
        elif alarm_dic[key] == 5 and pos_dic[key]==1:
            if float(current_price) < value[1]*1.08 and pre_price > value[1]*1.08:
                alarm(alarm_dic[key],name,'到达四级止赢价，止赢',current_price)
        elif alarm_dic[key] == 6 and pos_dic[key]==1:
            if float(current_price) < value[1]*1.08 and pre_price > value[1]*1.08:
                alarm(alarm_dic[key],name,'到达五级止赢价，止赢',current_price)
        
    
     
    time.sleep(10)

上海电影  价格：12.780  之前价格: 12.79  时间: 13:17:32


KeyboardInterrupt: 

In [167]:
import tushare as ts
import time
import pyttsx3
code_dic = {'601595':[12.25,12.10]}

for i in range(10):
    
    for key,value in code_dic.items():
        df = ts.get_realtime_quotes(key) #Single stock symbol
        dt = df[['code','name','price','bid','ask','volume','amount','time']]
        current_price = dt.price[0]
        name = dt.name[0]
        time1 = dt.time[0]
        print(name + '  ' + '价格：' + str(current_price) + '  时间: ' + str(time1))
        teacher = pyttsx3.init()
        teacher.say(name+'买入信号已到')
        teacher.runAndWait() 
    time.sleep(10)

上海电影  价格：12.170  时间: 14:03:25
上海电影  价格：12.180  时间: 14:03:37
上海电影  价格：12.180  时间: 14:03:46


KeyboardInterrupt: 

In [178]:
import pandas as pd
import numpy as np
de = ts.get_today_ticks('601595')#'000333'  002697
de

,time,price,vol,type
0,091502,12.61,16,-
1,091505,12.81,18,-
2,091514,12.87,20,-
3,091517,12.88,30,-
4,091635,12.91,36,-
...,...,...,...,...
570,100820,12.66,1,卖出
571,100823,12.65,38,买入
572,100827,12.66,11,卖出
573,100829,12.65,26,买入


({'601595': 0}, {'601595': [12.3, 12.19]})

In [166]:
df = ts.get_realtime_quotes('601595')
df

,name,open,pre_close,price,high,low,bid,ask,volume,amount,...,a2_p,a3_v,a3_p,a4_v,a4_p,a5_v,a5_p,date,time,code
0,上海电影,12.340,12.200,12.880,12.930,12.200,12.870,12.880,9672060,122404084.000,...,12.890,499,12.900,53,12.910,162,12.920,2021-03-16,15:00:02,601595


In [120]:
import akshare as ak
stock_zh_a_tick_163_df = ak.stock_zh_a_tick_163(code="sh600848")
# print(stock_zh_a_tick_163_df)

ValueError: File is not a recognized excel file

In [176]:
import dtshare as dt
stock_df = dt.get_tick_data("sh601595")
print(stock_df)#symbol="sh600000"

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


OSError: 获取失败，请检查网络.

In [152]:
df = pro.stk_mins(ts_code='600095.SH',start_time='2020-11-17 09:30:00',end_time='2020-11-17 15:00:00',freq='5min')
df

NameError: name 'pro' is not defined